# 📊 Financial Audit Data Analysis
This notebook explores and analyzes internal financial operations data.

In [1]:
import pandas as pd

# Load data from Excel
file_path = '../data/Financial_Audit_Dataset.xlsx'
vendors = pd.read_excel(file_path, sheet_name='Vendors')
po = pd.read_excel(file_path, sheet_name='PurchaseOrders')
invoices = pd.read_excel(file_path, sheet_name='Invoices')
ledger = pd.read_excel(file_path, sheet_name='LedgerEntries')

FileNotFoundError: [Errno 2] No such file or directory: '../data/Financial_Audit_Dataset.xlsx'

In [ ]:
# Display sample data
vendors.head()

In [ ]:
po.head()

In [ ]:
invoices.head()

In [ ]:
ledger.head()

## ✅ Next Steps:
- Match invoices with POs
- Detect unmatched or high-risk transactions
- Summarize department-level spending
- Visualize key metrics

## 🔍 Match Invoices with Purchase Orders

In [ ]:
# Merge invoices with purchase orders on PO_ID and VendorID
invoice_po_merged = invoices.merge(po, on=['PO_ID', 'VendorID'], how='left')

# Check for unmatched rows (i.e., invoices with no corresponding PO)
unmatched_invoices = invoice_po_merged[invoice_po_merged['Department'].isna()]
unmatched_invoices[['InvoiceID', 'PO_ID', 'VendorID', 'Amount_x', 'InvoiceDate']]

### 💡 Insight: These invoices have no matching purchase orders. They could be flagged for further audit review.

## 📊 High-Risk Vendor Analysis

In [ ]:
# Merge invoice data with vendor risk levels
invoice_vendor_merged = invoices.merge(vendors, on='VendorID', how='left')

# Filter for high-risk vendors
high_risk_vendors = invoice_vendor_merged[invoice_vendor_merged['RiskLevel'] == 'High']
high_risk_summary = high_risk_vendors.groupby('VendorName')['Amount'].sum().reset_index().sort_values(by='Amount', ascending=False)
high_risk_summary

### 📌 These are the vendors with the highest invoice totals and a high risk rating.

## 🏢 Department Spending Overview

In [ ]:
# Group total PO spending by department
department_spending = po.groupby('Department')['Amount'].sum().reset_index().sort_values(by='Amount', ascending=False)
department_spending

## 📈 Visualizations

In [ ]:
# Import visualization library
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

### 🧾 Department Spending Bar Chart

In [ ]:
# Plot department spending
plt.figure(figsize=(8,5))
sns.barplot(data=department_spending, x='Amount', y='Department', palette='Blues_d')
plt.title('Total Spending by Department')
plt.xlabel('Total Amount ($)')
plt.ylabel('Department')
plt.tight_layout()
plt.show()

## ⏱️ Payment Cycle Analysis

In [ ]:
# Calculate payment cycle (InvoiceDate - PO Date)
po['Date'] = pd.to_datetime(po['Date'])
invoices['InvoiceDate'] = pd.to_datetime(invoices['InvoiceDate'])
payment_merged = invoices.merge(po[['PO_ID', 'Date']], on='PO_ID', how='left')
payment_merged['PaymentCycleDays'] = (payment_merged['InvoiceDate'] - payment_merged['Date']).dt.days

# Show distribution
payment_merged[['InvoiceID', 'PO_ID', 'PaymentCycleDays']].head()

In [ ]:
# Plot distribution of payment cycles
plt.figure(figsize=(8,5))
sns.histplot(payment_merged['PaymentCycleDays'].dropna(), bins=20, kde=True)
plt.title('Distribution of Payment Cycles')
plt.xlabel('Days between PO and Invoice')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 🚨 Anomaly Detection: Outlier Payment Cycles

In [ ]:
# Flag payment cycles that are unusually long or short
mean_cycle = payment_merged['PaymentCycleDays'].mean()
std_cycle = payment_merged['PaymentCycleDays'].std()
payment_merged['Anomaly'] = payment_merged['PaymentCycleDays'].apply(
    lambda x: 'Outlier' if abs(x - mean_cycle) > 2 * std_cycle else 'Normal')

outliers = payment_merged[payment_merged['Anomaly'] == 'Outlier']
outliers[['InvoiceID', 'PO_ID', 'PaymentCycleDays']]

## 💰 Accounts Payable Turnover (APT) Analysis

We'll calculate the APT ratio using simulated procurement data and compare it against industry averages (e.g., 3.5–4.0 for the pharmaceutical industry).

In [ ]:
# Prepare monthly PO (purchases) and unpaid invoices (payables)
po['Date'] = pd.to_datetime(po['Date'])
invoices['InvoiceDate'] = pd.to_datetime(invoices['InvoiceDate'])

# Tag unpaid invoices
invoices['Unpaid'] = invoices['Status'] != 'Paid'

# Monthly total purchases
monthly_purchases = po.copy()
monthly_purchases['Month'] = monthly_purchases['Date'].dt.to_period('M')
purchase_summary = monthly_purchases.groupby('Month')['Amount'].sum().reset_index()

# Monthly unpaid balances (simulating AP ending balances)
unpaid_invoices = invoices[invoices['Unpaid']].copy()
unpaid_invoices['Month'] = unpaid_invoices['InvoiceDate'].dt.to_period('M')
ap_balances = unpaid_invoices.groupby('Month')['Amount'].sum().reset_index().rename(columns={'Amount': 'AP_Balance'})

In [ ]:
# Merge purchases and AP balances
apt_df = pd.merge(purchase_summary, ap_balances, on='Month', how='left')
apt_df['AP_Balance'] = apt_df['AP_Balance'].fillna(method='ffill')
apt_df['Avg_AP'] = apt_df['AP_Balance'].rolling(2).mean()
apt_df['APT'] = apt_df['Amount'] / apt_df['Avg_AP']
apt_df = apt_df.dropna()
apt_df.head()

In [ ]:
# Plot APT trend over time
plt.figure(figsize=(10,5))
sns.lineplot(data=apt_df, x='Month', y='APT', marker='o', label='Company APT')
plt.axhline(3.5, color='red', linestyle='--', label='Industry Avg (Pharma)')
plt.title('Accounts Payable Turnover Ratio Over Time')
plt.ylabel('APT')
plt.xlabel('Month')
plt.legend()
plt.tight_layout()
plt.show()

### 📌 Insight:
If your company's APT is consistently below the industry average (around 3.5–4.0), it may indicate slower payment cycles, which could affect vendor relationships or reflect credit leverage strategies.